### A notebook for running and visualizing results from
# The Tea Is A Lie
### - TTIAL: a Mesh Analysis Software for Steady State Diffusion 

#### available at Zenodo
#### DOI: XYZ

By Björn Stenqvist, Div. of Physical Chemistry, Lund University, Sweden
 
The following Python packages are needed to run the notebook:<br> numpy<br> matplotlib

In [ ]:
%matplotlib inline
import numpy as np, os
import matplotlib, matplotlib.pyplot as plt
from matplotlib import rc
import matplotlib as mpl

fontSizeLabel=10
fontSizeLegend=8
fontSizeAll=8
plt.rcParams.update({'font.size': fontSizeAll, 'figure.figsize': [3.513475, 2.713475],'xtick.labelsize':fontSizeAll,'ytick.labelsize':fontSizeAll})
rc('font',**{'family':'serif','sans-serif':['Times']}) # set font
#rc('text', usetex=True) # use tex (optional)
color_c='black' # color for reference
color_mu='black' # color for this work
insetColor='black' # color of lines in insets
cmaps = 'plasma'
R=8.31446261815324 # gas constant
T=298.15 # temperature
RT=R*T
mum=1e-6 # micrometer
! mkdir Output # create folder for output-files

def findValue(filename,key):
    with open (filename, "r") as hfile:
      sp = hfile.read()

    lines = sp.split("\n")
    for line in lines:
        words = line.split(" ")
        if words[0] == key:
            return words[1]
        
def getDimensions(filename):
    height = float(findValue(filename,'height'))
    width = float(findValue(filename,'width'))
    return height, width

## Generate input

In [ ]:
%%writefile input.txt
model_nbr     0     # model number, 0 gives Brick and Mortar
d             5.0   # width of bricks
d_comp        2.0   # width of complementary bricks
s             0.1   # horizontal spacing between bricks
g             0.1   # vertical spacing between bricks
t             1.0   # brick thickness
N             5     # nbr of layers of bricks
omega         1.0   # offset ratio, negative gives random
c_out         1.0   # concentration on one side of system (zero on the other)
S_out         1.0   # solubility outside system
S_mv          1e1   # mortar solubility (verticle)
S_mh          1e1   # mortar solubility (horizontal)
S_bv          1e-1  # brick solubility (verticle)
S_bh          1e-1  # brick solubility (horizontal)
D_mv          1.0   # mortar diffusion coefficient (verticle)
D_mh          1.0   # mortar diffusion coefficient (horizontal)
D_bv          1.0   # brick diffusion coefficient (verticle)
D_bh          1.0   # brick diffusion coefficient (horizontal)
seed          1     # seed, negative gives random
Nc            51    # number of columns of nodes in mesh
Nr            54    # number of rows of nodes in mesh
max_iter      1000    # maximum number of iterations when using conjugate gradient method, if negative then exact method is used
max_error     1e-3  # maximum (squared) error of the residual when using conjugate gradient method
output_folder Output # folder for output

## Run software

In [ ]:
! ../../ttial input.txt

## Visualize results

### Concentration

In [ ]:
def plotConcentrations():
    height, width = getDimensions('output.txt')
    fig, ((ax1),(ax2)) = plt.subplots(nrows=1, ncols=2)

    conc_ver = np.loadtxt('Output/conc_ver_matrix.txt')
    conc_hor = np.loadtxt('Output/conc_hor_matrix.txt')
    
    maxV = np.max([np.max(conc_ver),np.max(conc_hor)])
    minV = np.min([np.min(conc_ver),np.min(conc_hor)])
    
    M,N = np.shape(conc_ver)
    im = ax1.matshow(conc_ver,cmap=cmaps,vmin=minV,vmax=maxV)
    ax1.set_xticks([0,N/2,N-1])
    ax1.set_xticklabels(['0',str(width/2),str(width)])
    ax1.xaxis.set_ticks_position('bottom')
    ax1.set_yticks([0,M/2,M-1])
    ax1.set_yticklabels(['0',str(height/2),str(height)])
    ax1.tick_params(right=False,left=False,top=False,bottom=False)
    ax1.set_title('conc ver matrix', fontsize=12)
    ax1.set_xlabel('x', fontsize=12)
    ax1.set_ylabel('z', fontsize=12)

    M,N = np.shape(conc_hor)
    im2 = ax2.matshow(conc_hor,cmap=cmaps,vmin=minV,vmax=maxV)
    ax2.set_xticks([0,N/2,N-1])
    ax2.set_xticklabels(['0',str(width/2),str(width)])
    ax2.xaxis.set_ticks_position('bottom')
    ax2.set_yticklabels({})
    ax2.tick_params(right=False,left=False,top=False,bottom=False)
    ax2.set_title('conc hor matrix', fontsize=12)
    ax2.set_xlabel('x', fontsize=12)

    if minV < 0:
        print('Warning! Negative concentration')
    minV = 0
    cbticks = [minV,0.5*(minV+maxV),maxV]
    cblabels = [str(minV),str(0.5*(minV+maxV)),str(maxV)]
    cb_ax = fig.add_axes([1.0, 0.25, 0.04, 0.5])
    cbar = mpl.colorbar.ColorbarBase(cb_ax, ticks=cbticks, boundaries=np.linspace(minV,maxV,1000), cmap=cmaps,norm=mpl.colors.Normalize(vmin=minV, vmax=maxV))
    cbar.ax.set_yticklabels(cblabels, fontsize=12)
    plt.savefig('concentrations.pdf',dpi=300,bbox_inches='tight')
plotConcentrations()

### Flux

In [ ]:
def plotFlux():
    height, width = getDimensions('output.txt')
    fig, ((ax1),(ax2)) = plt.subplots(nrows=1, ncols=2)

    j_ver = np.loadtxt('Output/j_ver_matrix.txt')
    j_hor = np.loadtxt('Output/j_hor_matrix.txt')
    
    maxV = np.max([np.max(j_ver),np.max(j_hor)])
    minV = np.min([np.min(j_ver),np.min(j_hor)])
    
    M,N = np.shape(j_ver)
    im = ax1.matshow(j_ver,cmap=cmaps,vmin=minV,vmax=maxV)
    ax1.set_xticks([0,N/2,N-1])
    ax1.set_xticklabels(['0',str(width/2),str(width)])
    ax1.xaxis.set_ticks_position('bottom')
    ax1.set_yticks([0,M/2,M-1])
    ax1.set_yticklabels(['0',str(height/2),str(height)])
    ax1.tick_params(right=False,left=False,top=False,bottom=False)
    ax1.set_title('j ver matrix', fontsize=12)
    ax1.set_xlabel('x', fontsize=12)
    ax1.set_ylabel('z', fontsize=12)

    M,N = np.shape(j_hor)
    im2 = ax2.matshow(j_hor,cmap=cmaps,vmin=minV,vmax=maxV)
    ax2.set_xticks([0,N/2,N-1])
    ax2.set_xticklabels(['0',str(width/2),str(width)])
    ax2.xaxis.set_ticks_position('bottom')
    ax2.set_yticklabels({})
    ax2.tick_params(right=False,left=False,top=False,bottom=False)
    ax2.set_title('j hor matrix', fontsize=12)
    ax2.set_xlabel('x', fontsize=12)

    cbticks = [minV,0.5*(minV+maxV),maxV]
    cblabels = [str(minV),str(0.5*(minV+maxV)),str(maxV)]
    cb_ax = fig.add_axes([0.95, 0.25, 0.04, 0.5])
    cbar = mpl.colorbar.ColorbarBase(cb_ax, ticks=cbticks, boundaries=np.linspace(minV,maxV,1000), cmap=cmaps,norm=mpl.colors.Normalize(vmin=minV, vmax=maxV))
    cbar.ax.set_yticklabels(cblabels, fontsize=12)
    plt.savefig('flux_matrix.pdf',dpi=300,bbox_inches='tight')
    
    fig, (ax) = plt.subplots(nrows=1, ncols=1)
    j_ver = np.loadtxt('Output/j_ver_vector.txt')
    j_hor = np.loadtxt('Output/j_hor_vector.txt')
    ax.plot(j_ver[:,0],j_ver[:,1], lw=1, color='blue',ls='-',label=r'Vertical')
    ax.plot(j_hor[:,0],j_hor[:,1], lw=1, color='red',ls='-',label=r'Horizontal')
    ax.plot(np.linspace(0,height,2),[0,0], lw=1, color='black',ls='-')
    ax.set_xlim((0, height))
    ax.set_xlabel(r'Depth [m]', fontsize=12)
    ax.set_ylabel(r'Flux [kg/m$^3$]', fontsize=12)
    ax.legend(loc='center right', frameon=False,fontsize=12)
    plt.savefig('flux_vector.pdf',dpi=300,bbox_inches='tight')
plotFlux()

### Absolute activity

In [ ]:
def plotAbsoluteActivity():
    height, width = getDimensions('output.txt')
    fig, (ax1) = plt.subplots(nrows=1, ncols=1)

    absactivity = np.loadtxt('Output/V_matrix.txt')
    absactivity = np.kron(np.ones((1,2)),absactivity)
    
    M,N = np.shape(absactivity)
    im = ax1.matshow(absactivity,cmap=cmaps)
    ax1.set_xticks([0,N/2,N-1])
    ax1.set_xticklabels(['0',str(width/2),str(width)])
    ax1.xaxis.set_ticks_position('bottom')
    ax1.set_yticks([0,M/2,M-1])
    ax1.set_yticklabels(['0',str(height/2),str(height)])
    ax1.tick_params(right=False,left=False,top=False,bottom=False)
    ax1.set_title('V matrix', fontsize=12)
    ax1.set_xlabel('x', fontsize=12)
    ax1.set_ylabel('z', fontsize=12)

    maxV = np.max(absactivity)
    minV = np.min(absactivity)
    cbticks = [minV,0.5*(minV+maxV),maxV]
    cblabels = [str(minV),str(0.5*(minV+maxV)),str(maxV)]
    cb_ax = fig.add_axes([1.0, 0.25, 0.04, 0.5])
    cbar = mpl.colorbar.ColorbarBase(cb_ax, ticks=cbticks, boundaries=np.linspace(minV,maxV,1000), cmap=cmaps,norm=mpl.colors.Normalize(vmin=minV, vmax=maxV))
    cbar.ax.set_yticklabels(cblabels, fontsize=12)
    plt.savefig('absolute_activity.pdf',dpi=300,bbox_inches='tight')
plotAbsoluteActivity()

### Diffussion coefficient

In [ ]:
def plotDiffusionCoefficients():
    height, width = getDimensions('output.txt')
    fig, ((ax1),(ax2)) = plt.subplots(nrows=1, ncols=2)

    diffcoeff_ver = np.loadtxt('Output/Dv_matrix.txt')
    diffcoeff_hor = np.loadtxt('Output/Dh_matrix.txt')
    
    maxV = np.max([np.max(diffcoeff_ver),np.max(diffcoeff_hor)])
    minV = np.min([np.min(diffcoeff_ver),np.min(diffcoeff_hor)])
    
    M,N = np.shape(diffcoeff_ver)
    im = ax1.matshow(diffcoeff_ver,cmap=cmaps,vmin=minV,vmax=maxV)
    ax1.set_xticks([0,N/2,N-1])
    ax1.set_xticklabels(['0',str(width/2),str(width)])
    ax1.xaxis.set_ticks_position('bottom')
    ax1.set_yticks([0,M/2,M-1])
    ax1.set_yticklabels(['0',str(height/2),str(height)])
    ax1.tick_params(right=False,left=False,top=False,bottom=False)
    ax1.set_title('Dv matrix', fontsize=12)
    ax1.set_xlabel('x', fontsize=12)
    ax1.set_ylabel('z', fontsize=12)


    M,N = np.shape(diffcoeff_hor)
    im2 = ax2.matshow(diffcoeff_hor,cmap=cmaps,vmin=minV,vmax=maxV)
    ax2.set_xticks([0,N/2,N-1])
    ax2.set_xticklabels(['0',str(width/2),str(width)])
    ax2.xaxis.set_ticks_position('bottom')
    ax2.set_yticklabels({})
    ax2.tick_params(right=False,left=False,top=False,bottom=False)
    ax2.set_title('Dh matrix', fontsize=12)
    ax2.set_xlabel('x', fontsize=12)


    if minV < 0:
        print('Warning! Negative diffusion coefficients')
    cbticks = [minV,0.5*(minV+maxV),maxV]
    cblabels = [str(minV),str(0.5*(minV+maxV)),str(maxV)]
    cb_ax = fig.add_axes([0.95, 0.25, 0.04, 0.5])
    cbar = mpl.colorbar.ColorbarBase(cb_ax, ticks=cbticks, boundaries=np.linspace(minV,maxV,1000), cmap=cmaps,norm=mpl.colors.Normalize(vmin=minV, vmax=maxV))
    cbar.ax.set_yticklabels(cblabels, fontsize=12)
    plt.savefig('diffusion_coefficients.pdf',dpi=300,bbox_inches='tight')
plotDiffusionCoefficients()

### Solubility

In [ ]:
def plotSolubility():
    height, width = getDimensions('output.txt')
    fig, ((ax1),(ax2)) = plt.subplots(nrows=1, ncols=2)

    s_ver = np.loadtxt('Output/sv_matrix.txt')
    s_hor = np.loadtxt('Output/sh_matrix.txt')
    
    maxV = np.max([np.max(s_ver),np.max(s_hor)])
    minV = np.min([np.min(s_ver),np.min(s_hor)])
    
    M,N = np.shape(s_ver)
    im = ax1.matshow(s_ver,cmap=cmaps,vmin=minV,vmax=maxV)
    ax1.set_xticks([0,N/2,N-1])
    ax1.set_xticklabels(['0',str(width/2),str(width)])
    ax1.xaxis.set_ticks_position('bottom')
    ax1.set_yticks([0,M/2,M-1])
    ax1.set_yticklabels(['0',str(height/2),str(height)])
    ax1.tick_params(right=False,left=False,top=False,bottom=False)
    ax1.set_title('sv matrix', fontsize=12)
    ax1.set_xlabel('x', fontsize=12)
    ax1.set_ylabel('z', fontsize=12)

    M,N = np.shape(s_hor)
    im2 = ax2.matshow(s_hor,cmap=cmaps,vmin=minV,vmax=maxV)
    ax2.set_xticks([0,N/2,N-1])
    ax2.set_xticklabels(['0',str(width/2),str(width)])
    ax2.xaxis.set_ticks_position('bottom')
    ax2.set_yticklabels({})
    ax2.tick_params(right=False,left=False,top=False,bottom=False)
    ax2.set_title('sh matrix', fontsize=12)
    ax2.set_xlabel('x', fontsize=12)

    cbticks = [minV,0.5*(minV+maxV),maxV]
    cblabels = [str(minV),str(0.5*(minV+maxV)),str(maxV)]
    cb_ax = fig.add_axes([0.95, 0.25, 0.04, 0.5])
    cbar = mpl.colorbar.ColorbarBase(cb_ax, ticks=cbticks, boundaries=np.linspace(minV,maxV,1000), cmap=cmaps,norm=mpl.colors.Normalize(vmin=minV, vmax=maxV))
    cbar.ax.set_yticklabels(cblabels, fontsize=12)
    plt.savefig('solubility.pdf',dpi=300,bbox_inches='tight')
plotSolubility()